In [13]:
# Import

import pandas as pd
import numpy as np
import os

import sqlalchemy
import datetime
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float, DateTime

In [5]:
# Read measurements CSV

clean_measurements = pd.read_csv('clean_measurements.csv')
clean_measurements.head(10)

,Unnamed: 0,station,date,prcp,tobs
0,0,USC00519397,2010-01-01,0.08,65
1,5,USC00519397,2010-01-07,0.06,70
2,9,USC00519397,2010-01-11,0.01,64
3,21,USC00519397,2010-01-24,0.01,71
4,23,USC00519397,2010-01-26,0.04,76
5,24,USC00519397,2010-01-27,0.12,68
6,27,USC00519397,2010-01-31,0.03,67
7,28,USC00519397,2010-02-01,0.01,66
8,30,USC00519397,2010-02-04,0.01,69
9,37,USC00519397,2010-02-12,0.02,69


In [10]:
# Change column name 'Unnamed:0'

clean_measurements.rename(columns={'Unnamed: 0': 'id'}, inplace=True)
clean_measurements.to_csv('clean_measurements_2.csv')
clean_measurements.head()

,id,station,date,prcp,tobs
0,0,USC00519397,2010-01-01,0.08,65
1,5,USC00519397,2010-01-07,0.06,70
2,9,USC00519397,2010-01-11,0.01,64
3,21,USC00519397,2010-01-24,0.01,71
4,23,USC00519397,2010-01-26,0.04,76


In [12]:
clean_stations = pd.read_csv('clean_stations.csv')
clean_stations.rename(columns={'Unnamed: 0': 'id'}, inplace=True)
clean_stations.to_csv('clean_stations_2.csv')
clean_stations.head()

,id,station,name,latitude,longitude,elevation
0,0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


In [15]:
# Create SQLite engine

engine = create_engine('sqlite:///hawaii.sqlite')
conn = engine.connect()

In [17]:
# Create Measurements Table

Base = declarative_base()

class Measurement(Base):
    __tablename__ = 'measurement'
    
    id = Column(Integer, primary_key=True)
    station = Column(String)
    date = Column(String)
    prcp = Column(Float)
    tobs = Column(Float)
    
        
    def __repr__(self):
        return f"id={self.id}, station={self.station}"

In [19]:
# Create Station Table

class Station(Base):
    __tablename__ = 'station'
    
    id = Column(Integer, primary_key = True)
    station = Column(String)
    name = Column(String)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
    
    def __repr__(self):
        return f"id={self.id}, name={self.name}"

C:\Users\JChavez\AppData\Local\Continuum\anaconda3\lib\site-packages\sqlalchemy\ext\declarative\clsregistry.py:120: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.Station, and will be replaced in the string-lookup table.
  item.__name__


InvalidRequestError: Table 'station' is already defined for this MetaData instance.  Specify 'extend_existing=True' to redefine options and columns on an existing Table object.

In [20]:
Base.metadata.create_all(engine)

In [22]:
stations_df = pd.read_csv('clean_stations_2.csv')
meas_df = pd.read_csv('clean_measurements_2.csv')

In [23]:
stations_data = stations_df.to_dict(orient='records')
meas_data = meas_df.to_dict(orient='records')

In [24]:
metadata = MetaData(bind=engine)
metadata.reflect()

In [25]:
table_stations = sqlalchemy.Table('station', metadata, autoload=True)
table_meas = sqlalchemy.Table('measurement', metadata, autoload=True)

In [26]:
table_meas

Table('measurement', MetaData(bind=Engine(sqlite:///hawaii.sqlite)), Column('id', INTEGER(), table=<measurement>, primary_key=True, nullable=False), Column('station', VARCHAR(), table=<measurement>), Column('date', VARCHAR(), table=<measurement>), Column('prcp', FLOAT(), table=<measurement>), Column('tobs', FLOAT(), table=<measurement>), schema=None)

In [27]:
table_stations

Table('station', MetaData(bind=Engine(sqlite:///hawaii.sqlite)), Column('id', INTEGER(), table=<station>, primary_key=True, nullable=False), Column('station', VARCHAR(), table=<station>), Column('name', VARCHAR(), table=<station>), Column('latitude', FLOAT(), table=<station>), Column('longitude', FLOAT(), table=<station>), Column('elevation', FLOAT(), table=<station>), schema=None)

In [28]:
conn.execute(table_stations.delete())

In [29]:
conn.execute(table_meas.delete())